In [87]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

def get_text_by_regex(isolated_tag,regex_text):
    return str(isolated_tag.find(text=re.compile(regex_text)))

def get_attrib_text_by_regex(isolated_tag,regex_text,att_name):
    return str(isolated_tag.find(attrs={att_name :re.compile(regex_text)}).attrs[att_name])

def expand_youtube_video_link(url):

    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--start-maximized")
#     options.add_argument("--headless")
    driver = webdriver.Chrome(options=options,executable_path='./driver/chromedriver.exe')
    wait = WebDriverWait(driver,60)
    driver.get(url)
    SCROLL_PAUSE_TIME = 2
    CYCLES = 7
    html = driver.find_element_by_tag_name('html')
   
    html.send_keys(Keys.PAGE_DOWN)  
    time.sleep(SCROLL_PAUSE_TIME)
    html.send_keys(Keys.PAGE_DOWN)  
    time.sleep(SCROLL_PAUSE_TIME)
    TOTAL_COMMENT=driver.find_elements_by_xpath('//*[@id="header"]/ytd-comments-header-renderer')#'/div[@id="header"]')
    
    #expanding video descriptions show more button
    show_more=driver.find_element_by_xpath('//*[@id="more"]/yt-formatted-string')
    show_more.location_once_scrolled_into_view
    driver.execute_script("arguments[0].click()",show_more)
    
    html.send_keys(Keys.PAGE_DOWN)  
    time.sleep(SCROLL_PAUSE_TIME)
    html.send_keys(Keys.PAGE_DOWN)  
    time.sleep(SCROLL_PAUSE_TIME)
    
#     print([x.text for x in TOTAL_COMMENT])
    TOTAL_COMMENT=[x.text for x in TOTAL_COMMENT if "comments" in x.text.lower()]
    if TOTAL_COMMENT:
        TOTAL_COMMENT=int(TOTAL_COMMENT[0].split()[0].replace(",",""))
    else:
        TOTAL_COMMENT=0 
    print(TOTAL_COMMENT)
    
    VISIBLECOUNTS = len(driver.find_elements_by_xpath('//*[@id="contents"]/ytd-comment-thread-renderer'))
#     print(VISIBLECOUNTS)
    time.sleep(SCROLL_PAUSE_TIME * 3)
    i=0
    while VISIBLECOUNTS  < TOTAL_COMMENT:
        html.send_keys(Keys.END)
        time.sleep(SCROLL_PAUSE_TIME)
        VISIBLECOUNTS = len(driver.find_elements_by_xpath('//*[@id="contents"]/ytd-comment-thread-renderer'))
        #remove the break statement to expand to all comment or add your own custom break statement (on certain on of visible comment)
        i+=1
        if i==10:
            break
    for reply in driver.find_elements_by_xpath("//*[@id='replies']//paper-button[@class='style-scope ytd-button-renderer'][contains(.,'View')]"):
#         print('in for loop')
        reply.location_once_scrolled_into_view
        driver.execute_script("arguments[0].click()",reply)
    p_source=driver.page_source
    driver.close()
    return p_source

In [82]:
url='https://www.youtube.com/watch?v=yNaN5kYTNLY&ab_channel=warikoo'
page_source = expand_youtube_video_link(url)
soup = BeautifulSoup(page_source,"html.parser")

['4,597 Comments\nSORT BY\nAdd a public comment...']
4597


In [83]:
#get video information.
video_info_tag=soup.find('div',{"class":"style-scope ytd-video-primary-info-renderer"})

print("Video Title:")
video_title=soup.find('yt-formatted-string',{"class":"style-scope ytd-video-primary-info-renderer"}).text
print(video_title)

#get views
views_regex=r'\b[0-9]*views\b'
views=get_text_by_regex(video_info_tag,views_regex)
print('\n'*2)
print("View Count:")
print(views)

video_date_regex=r'\b[A-Z][a-z]*[\. | ][0-9]{2}, [0-9]{4}\b'
date_published=get_text_by_regex(video_info_tag,video_date_regex)
print('\n'*2)
print("Publish Date:")
print(date_published)


likes_count=get_attrib_text_by_regex(video_info_tag,r'\b[0-9]*likes\b','aria-label')
print('\n'*2)
print("Like Count:")
print(likes_count)


dislikes_count=get_attrib_text_by_regex(video_info_tag,r'\b[0-9]*dislikes\b','aria-label')
print('\n'*2)
print("Dislike Count:")
print(dislikes_count)

secondary_video_info_tag=soup.find('div',{"class":"style-scope ytd-video-secondary-info-renderer"})
sub_count=get_attrib_text_by_regex(secondary_video_info_tag,r'\b[0-9]*subscribers\b','aria-label')
print('\n'*2)
print("Subscribers count:")
print(sub_count)

channel_name=secondary_video_info_tag.find('yt-formatted-string',{"class":"style-scope ytd-channel-name"}).text
print('\n'*2)
print("Channel Name:")
print(channel_name)

channel_link=secondary_video_info_tag.find('yt-formatted-string',{"class":"style-scope ytd-channel-name"}).find('a').attrs['href']
print('\n'*2)
print("Channel Link:")
print("https://www.youtube.com"+channel_link)


video_description=secondary_video_info_tag.find('div',{"id":"description"}).text
print('\n'*2)
print("Video Description:")
print(video_description)

# aria-label

Video Title:
The best ways to Invest in your 20s | Investing tips for Beginners 2021 |  Ankur Warikoo Hindi



View Count:
784,003 views



Publish Date:
Jan 19, 2021



Like Count:
43,841 likes



Dislike Count:
541 dislikes



Subscribers count:
291K subscribers



Channel Name:
warikoo
https://www.youtube.com/channel/UCRzYN32xtBf3Yxsx5BvJWJw



Video Description:
How to invest in your 20s?
How to pick the right investment option?
What are the top investing mistakes to avoid?

I use Zerodha to invest in stocks. You can start your investing journey too, through Zerodha.
https://zerodha.com/open-account?c=ZM...​
(PS: When you create a Zerodha account through this link, I get an affiliate fee. There is no additional charge to you)

smallcase All-Weather-Investing (what I recommend for all of you, if you are just started investing in stocks): https://link.smallcase.com/eBDwWqpgocb​
#investmenttips​ #wealthcreation​ #warikoo​

In this video, I will talk to you about the order of preferenc

In [84]:
i=0
comments_list=[]
for element in soup.find_all('ytd-comment-thread-renderer'):

    temp_list=[url]
    comment=element.find('ytd-comment-renderer')
    author=comment.find('a',{"id":"author-text"})
    print(author.text.strip())
    temp_list.append(author.text.strip())
    publish_time=comment.find('yt-formatted-string',{"class":"published-time-text above-comment style-scope ytd-comment-renderer"})#
    print(publish_time.text.strip())
    temp_list.append(publish_time.text.strip())
    comment_text=comment.find('ytd-expander',{"class":"style-scope ytd-comment-renderer"}).find('div',{"id":"content"})
    print(comment_text.text.strip(),"---")
    temp_list.append(comment_text.text.strip())
    #vote-count-middle
    up_vote=comment.find('span',{"id":"vote-count-middle"})
    print(up_vote.text.strip(),"---")
    temp_list.append(up_vote.text.strip())
    replies=element.find('div', {"id":"replies"})


    comments_list.append(temp_list)

Bathmavathi Krishnan
2 days ago
Bitcoin is designed to be a huge step forward in making money more secure and could also act as a significant protection against any forms of financial crime ---
59 ---
Nikhil Suresh
3 weeks ago
kal ka exam se zyada important ye laga ---
423 ---
Arc Reactor
1 week ago
I thought "so Bruce Banner is doing this after endgame" for a sec 😂 ---
101 ---
kunal kedia
2 weeks ago
1) Don't take loans in your 20s
2)Take life and health insurance earliest.
3) Invest as much as you can, at least 20% of your income.(Time for compounding)
4)Invest in equities and stocks. ---
243 ---
Trisha Patel
4 weeks ago
Passive income is the key to unlock financial freedom. ---
461 ---
Arnab Mitra
2 months ago
If a teen or 20 year old has started thinking about investing, he has already won in life. Good luck 👍 ---
3K ---
NathanBalakrishna
1 month ago
This guy has the best skin i have ever seen. ---
248 ---
Ayush Hegde
4 days ago (edited)
3:25 When Ankur reveals his monthly income b

Prasanshita Singh
4 weeks ago
plzz make more videos on investing in 20s ---
0 ---
Sachin Jaiswal : Your Digital Expert
3 weeks ago
Great advice sir, you are best👌, but initially I saw your video I thought it's Bruce banner from avenger 😅, still awesome & helpful content sir. ---
0 ---
Adnan Masih
1 week ago
Very nice 👌 u just change my way of thinking. ---
0 ---
Bhupesh Kharkwal
2 months ago (edited)
Please make review on "The psychology of Money" Book.... ---
29 ---
Aman Jain
3 weeks ago
Amazing video. Crisp and clear. ---
0 ---
Hello 2 World
6 days ago
Great information! Hey, btw, did you consider the tax on interest? ---
0 ---
Great people of HIMACHAL
4 hours ago
Baaki sab toh theek hai
But
Invest kaha karna hai aur kaise karna hai aur kis pe karna hai 

Yee toh bataya he nahi🤦🏼‍♂️ ---
1 ---
SᗩᗰEEᖇ ᕼᗩIᗪEᖇ
2 days ago
Hulk Bhaiya 🙌 ---
1 ---
Vishwanand
2 months ago
You deserve millions subscribers. ---
3 ---
Aviral Singh
1 week ago
Hello Ankur, it would be of great help if you could m

In [85]:
cols=['video_url','comment_author','publish_time','comment_text','up_vote']
df=pd.DataFrame(comments_list,columns=cols)

In [86]:
df

,video_url,comment_author,publish_time,comment_text,up_vote
0,https://www.youtube.com/watch?v=yNaN5kYTNLY&ab...,Bathmavathi Krishnan,2 days ago,Bitcoin is designed to be a huge step forward ...,59
1,https://www.youtube.com/watch?v=yNaN5kYTNLY&ab...,Nikhil Suresh,3 weeks ago,kal ka exam se zyada important ye laga,423
2,https://www.youtube.com/watch?v=yNaN5kYTNLY&ab...,Arc Reactor,1 week ago,"I thought ""so Bruce Banner is doing this after...",101
3,https://www.youtube.com/watch?v=yNaN5kYTNLY&ab...,kunal kedia,2 weeks ago,1) Don't take loans in your 20s\n2)Take life a...,243
4,https://www.youtube.com/watch?v=yNaN5kYTNLY&ab...,Trisha Patel,4 weeks ago,Passive income is the key to unlock financial ...,461
...,...,...,...,...,...
215,https://www.youtube.com/watch?v=yNaN5kYTNLY&ab...,Abhilash putti,2 months ago,Absolutely! Live like a pauper in 20s. Best ad...,4
216,https://www.youtube.com/watch?v=yNaN5kYTNLY&ab...,Amrit Bhat,3 weeks ago,18000?,0
217,https://www.youtube.com/watch?v=yNaN5kYTNLY&ab...,Akshat Sharma,1 week ago,"Sir , a big thanks for this amazingly informat...",1
218,https://www.youtube.com/watch?v=yNaN5kYTNLY&ab...,Anand Singh,3 weeks ago,I'm 21 and i am a garments manufacturer... any...,0
